In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import os
import pandas as pd
import time

import win32com.client as win32 
from pathlib import Path

In [ ]:
company_df = pd.read_excel('客戶資料-公發以上.xlsx')
com_code_ls = company_df["stock_num"].to_list()


# # GET remaining values
# index = com_code_ls.index("6913")

# # slice list to get remaining values
# remaining_values = com_code_ls[index+1:]
# print(com_code_ls)

### Download XBRL html file from [MOPS 公開資訊觀測站](https://mops.twse.com.tw/mops/web/t203sb01)

In [ ]:
driver = webdriver.Chrome()
driver.get("https://mops.twse.com.tw/mops/web/t203sb01")
company_code = driver.find_element(By.ID, "co_id")


# for company in remaining_values:
num = driver.find_element(By.ID,'co_id')
# num.send_keys(f'{company}')
num.send_keys('000218')

search = driver.find_element(By.XPATH,'/html/body/center/table/tbody/tr/td/div[4]/table/tbody/tr/td/div/table/tbody/tr/td[3]/div/div[3]/form/table/tbody/tr/td[4]/table/tbody/tr/td[2]/div/div/input')
driver.execute_script('arguments[0].click();', search)
soup = BeautifulSoup(driver.page_source,'html.parser')

time.sleep(5)

tds = driver.find_elements(By.TAG_NAME, 'td')
for i in range(len(tds)):

    if tds[i].text == '110Q4' and tds[i+1].text == '合併':
        download = driver.find_element(By.XPATH, f'/html/body/center/table/tbody/tr/td/div[4]/table/tbody/tr/td/div/table/tbody/tr/td[3]/div/div[5]/div/form/table/tbody/tr[4]/td[4]/input')
        driver.execute_script('arguments[0].click();', download)
    
    elif tds[i].text == '111Q2' and tds[i+1].text == '合併':
        download = driver.find_element(By.XPATH, '/html/body/center/table/tbody/tr/td/div[4]/table/tbody/tr/td/div/table/tbody/tr/td[3]/div/div[5]/div/form/table/tbody/tr[3]/td[4]/input')
        driver.execute_script('arguments[0].click();', download)
    
    else:
        pass

    num.clear()
time.sleep(8)

driver.close()

### Retrieve Desired Data from the html files

In [4]:
dir = 'XBRL_file'
for filename in os.listdir(dir):
    fname = os.path.join(dir,filename)
    with open(fname, 'r', encoding="utf-8") as f:
        soup = BeautifulSoup(f,'html.parser')

        # Find the table with the desired th tag
        th_tag_text = "列入合併財務報表之子公司"
        table = None
        for th in soup.find_all("th"):
            if th.text.strip() == th_tag_text:
                table = th.find_parent("table")
                break
        
        header_str = soup.find_all('div', class_='header')[0].select_one("span.zh").text
        start_index = header_str.index(' ')
        if "2021年" in header_str:
            end_index = header_str.index('2021年')
        elif "2022年" in header_str:
            end_index = header_str.index('2022年')
        company_name = header_str[start_index : end_index].strip()
        stock_number = header_str[:start_index]

    
        # info_dict = company_df[company_df['stock_num'] == stock_number].to_dict('records')
        # audit_team = info_dict[0]['team']
        # client_code = info_dict[0]['client_code']
    

        # Convert the table to a Pandas DataFrame
        if table is not None:
            df = pd.read_html(str(table), flavor='bs4')[0]
            df.columns = df.columns.droplevel([0, 1]) #remove first two levels
            appended_values = pd.concat([df['投資公司名稱'], df['子公司名稱']]).unique()

            df = pd.DataFrame({'Company': appended_values, '查核事務所':'', 'Client Code':'', '公司類型':'', '關係':''})
            exclude_list = ['註1', '備註', '本 公 司', '註：', '本公司', '註:'] # Define the values to exclude
            mask = df['Company'].str.contains('|'.join(exclude_list))
            
            df['Company'] = df['Company'].str.replace("(註)", '', regex=False) #remove "(註)"

            # df[~mask].to_excel(f'Result/{audit_team}_{client_code}_{stock_number}_{company_name}.xlsx', index=False)
            df[~mask].to_excel(f'Result/{stock_number}_{company_name}.xlsx', index=False)
            
        else:
            df = pd.DataFrame({'Company': company_name, '查核事務所':'', 'Client Code':'', '公司類型':'', '關係':''}, index=[0])
            # df.to_excel(f'Result/{audit_team}_{client_code}_{stock_number}_{company_name}.xlsx', index=False)
            df.to_excel(f'Result/{stock_number}_{company_name}.xlsx', index=False)

TypeError: bad operand type for unary ~: 'float'

In [120]:
for i in range(len(company_df)):
    stk_num, team, client_c, client_n = company_df['stock_num'][i], company_df['team'][i], company_df['client_code'][i], company_df['客戶名稱'][i]
    df_indiv = pd.DataFrame({'Company': client_n, '查核事務所':'', 'Client Code':'', '公司類型':'', '關係':''}, index=[0])
    df_indiv.to_excel(f'Result2/{team}_{client_c}_{stk_num}_{client_n}.xlsx', index=False)

### VBA

In [121]:
excel = win32.gencache.EnsureDispatch('Excel.Application')

# excel can be visible or not
excel.Visible = False

# open workbooks
f_path = Path.cwd()
f_path2 = f_path /'Result/' 

template_f = 'A10A_AMI002_5272_笙科電子股份有限公司.xlsx'
# template = f_path / template_f

for filename in os.listdir(f_path2):
    fname = os.path.join(f_path2,filename)

    template_wb = excel.Workbooks.Open(template_f)
    target_wb = excel.Workbooks.Open(fname)
#1. copy and paste "集團公費邏輯指標 3-1 非審計服務公費說明" sheet to each file
    template_wb.Worksheets("集團公費邏輯指標 3-1 非審計服務公費說明").Copy(Before=target_wb.Sheets(1))

#2. copy and paste drop down list (data validation) to each file
    dropdown_sheet = template_wb.Worksheets("Sheet1")
    target_sheet = target_wb.Worksheets("Sheet1")
    rowCnt_f = target_sheet.UsedRange.Rows.Count
    for i in range(2, rowCnt_f+1):
        dropdown_sheet.Range("B2:E2").Copy()
        target_sheet.Range("B%s:E%s"%(i,i)).PasteSpecial(Paste=-4104)

#3. set column width and font
    info_ws = target_wb.Worksheets("集團公費邏輯指標 3-1 非審計服務公費說明")
    info_ws.Range("A:B").Font.Name = "Calibri"
    target_sheet.Range("A:A").ColumnWidth = 100
    target_sheet.Range("B:E").ColumnWidth = 15
    target_sheet.Range("A:E").Font.Name = "Calibri"


    # remove hashtag to save and quit modified Excel file
    target_wb.Save()
    target_wb.Close(True)
excel.Quit()